In [3]:
#OBJECT DETECTION

from ultralytics import YOLO
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from PIL import Image

# —— CONFIG ——
weights    = "runs/detect/train35/weights/best.pt"
val_root   = "Road_Damage/val"
folders    = ["potholes", "crack_issues", "alligator_crack_issues", "ravelling", "pumping_and_depression", "open_manhole"] 
img_sub    = "images"
lbl_sub    = "labels"
iou_thresh = 0.5
out_path   = "det_confusion_matrix.png"

# —— HELPERS ——
def load_yolo_txt(label_path):
    boxes = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            cls = int(parts[0])
            coords = list(map(float, parts[1:5]))
            boxes.append([cls, *coords])
    return np.array(boxes)

def xywh_to_xyxy(box, img_w, img_h):
    cx, cy, w, h = box
    x1 = (cx - w/2) * img_w
    y1 = (cy - h/2) * img_h
    x2 = (cx + w/2) * img_w
    y2 = (cy + h/2) * img_h
    return [x1, y1, x2, y2]

def iou(boxA, boxB):
    xA1,yA1,xA2,yA2 = boxA
    xB1,yB1,xB2,yB2 = boxB
    xi1 = max(xA1, xB1); yi1 = max(yA1, yB1)
    xi2 = min(xA2, xB2); yi2 = min(yA2, yB2)
    inter = max(0, xi2-xi1) * max(0, yi2-yi1)
    areaA = (xA2-xA1)*(yA2-yA1)
    areaB = (xB2-xB1)*(yB2-yB1)
    union = areaA + areaB - inter
    return inter/union if union > 0 else 0

# —— LOAD MODEL & NAMES ——
model = YOLO(weights)
names = model.names             # e.g. {0:"crack_issues",1:"potholes"}
class_idx_list = sorted(names.keys())

# —— MATCH & COLLECT ——
y_true, y_pred = [], []

for folder in folders:
    img_dir = os.path.join(val_root, folder, img_sub)
    lbl_dir = os.path.join(val_root, folder, lbl_sub)
    for img_file in os.listdir(img_dir):
        if not img_file.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        img_path = os.path.join(img_dir, img_file)
        lbl_path = os.path.join(lbl_dir, img_file.rsplit(".",1)[0] + ".txt")
        if not os.path.isfile(lbl_path):
            continue

        # Load and convert GT boxes
        gt = load_yolo_txt(lbl_path)
        w, h = Image.open(img_path).size
        gt_boxes = []
        for cls, x, y, wd, hd in gt:
            xyxy = xywh_to_xyxy([x, y, wd, hd], w, h)
            gt_boxes.append([int(cls), *xyxy])
        gt_boxes = np.array(gt_boxes)

        # Run detection
        res = model(img_path, imgsz=640)[0]
        preds_xyxy = res.boxes.xyxy.cpu().numpy()
        preds_cls  = res.boxes.cls.cpu().numpy().astype(int)

        matched_gt = set()
        # Greedy match preds → GT
        for pi, pred_box in enumerate(preds_xyxy):
            best_iou = 0.0
            best_gi  = -1
            for gi, gt_box in enumerate(gt_boxes):
                if gi in matched_gt:
                    continue
                i = iou(pred_box, gt_box[1:])
                if i > best_iou:
                    best_iou, best_gi = i, gi
            if best_iou >= iou_thresh and best_gi >= 0:
                y_true.append(int(gt_boxes[best_gi, 0]))
                y_pred.append(int(preds_cls[pi]))
                matched_gt.add(best_gi)

# —— BUILD & SAVE CONFUSION MATRIX ——
cm   = confusion_matrix(y_true, y_pred, labels=class_idx_list)
disp = ConfusionMatrixDisplay(cm, display_labels=[names[i] for i in class_idx_list])

fig, ax = plt.subplots(figsize=(7, 6))
disp.plot(ax=ax, cmap="Blues", xticks_rotation="vertical")
plt.title("Detection Confusion Matrix (IoU ≥ 0.5)")
plt.tight_layout()
plt.savefig(out_path, dpi=150)
plt.close(fig)

print(f"Saved detection confusion matrix to {out_path}")



image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val\potholes\images\01 - Chuck HolePunch Out (6).jpg: 640x640 2 potholess, 24.1ms
Speed: 4.3ms preprocess, 24.1ms inference, 131.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val\potholes\images\01 - Chuck-holePunch-out (RL).jpg: 640x640 3 potholess, 74.0ms
Speed: 8.4ms preprocess, 74.0ms inference, 5.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val\potholes\images\01 - Patholes.jpg: 640x640 2 potholess, 12.5ms
Speed: 2.3ms preprocess, 12.5ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val\potholes\images\01 - pothole (4).jpg: 640x640 1 potholes, 23.5ms
Speed: 2.3ms preprocess, 23.5ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val\potho

In [2]:
#CLASSIFICATION

from ultralytics import YOLO
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import os

# 1. Reload your trained weights
weights = "runs/classify/train20/weights/best.pt"
model   = YOLO(weights)

# 2. Grab the model's own names dict
#    e.g. {0: "crack_issues", 1: "potholes"}
names = model.names  

# 3. Validation root and your folder names
val_root = "Road_Damage/val"
folders  = ["potholes", "crack_issues", "alligator_crack_issues", "ravelling", "pumping_and_depression", "open_manhole"]  # whatever sub-folders you have

# 4. Invert names → index for easy lookup
name_to_idx = {v: k for k, v in names.items()}

y_true, y_pred = [], []

for folder in folders:
    true_idx = name_to_idx[folder]
    img_dir  = os.path.join(val_root, folder, "images")
    for img in os.listdir(img_dir):
        if not img.lower().endswith((".jpg", ".jpeg", ".png")):
            continue
        path = os.path.join(img_dir, img)
        y_true.append(true_idx)
        res = model(path, imgsz=640)[0]
        y_pred.append(int(res.probs.top1))

# 5. Print classification report
print("\n📊 Classification Report:")
print(classification_report(
    y_true, 
    y_pred, 
    target_names=[names[i] for i in sorted(name_to_idx.values())],
    zero_division=0
))

# 5. Build & save the confusion matrix
cm   = confusion_matrix(y_true, y_pred, labels=sorted(name_to_idx.values()))
disp = ConfusionMatrixDisplay(cm,
       display_labels=[names[i] for i in sorted(name_to_idx.values())])

fig, ax = plt.subplots(figsize=(7,6))
disp.plot(ax=ax, cmap="Blues", xticks_rotation="vertical")
plt.title("Validation Confusion Matrix")
plt.tight_layout()

# **Save** to file instead of (or as well as) plt.show()
out_path = "confusion_matrix.png"
plt.savefig(out_path, dpi=150)
print(f"Saved confusion matrix to {out_path}")
plt.close(fig)


image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val\potholes\images\01 - Chuck HolePunch Out (6).jpg: 640x640 potholes 1.00, crack_issues 0.00, open_manhole 0.00, pumping_and_depression 0.00, alligator_crack_issues 0.00, 7.4ms
Speed: 6.3ms preprocess, 7.4ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val\potholes\images\01 - Chuck-holePunch-out (RL).jpg: 640x640 potholes 1.00, crack_issues 0.00, pumping_and_depression 0.00, open_manhole 0.00, ravelling 0.00, 11.1ms
Speed: 7.4ms preprocess, 11.1ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jazzb\ImageDetection-Yolov9\Road_Damage\val\potholes\images\01 - Patholes.jpg: 640x640 ravelling 0.95, crack_issues 0.03, pumping_and_depression 0.02, potholes 0.01, alligator_crack_issues 0.00, 5.7ms
Speed: 7.1ms preprocess, 5.7ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jaz